DATA2001 Group Assignment <br>
Task 1: Import and Clean <br>
Package Imports 

In [1]:
# package load
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.wkt import loads
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

In [2]:
sa2 = gpd.read_file('data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
future = gpd.read_file('data/catchments/catchments_future.shp')
primary = gpd.read_file('data/catchments/catchments_primary.shp')
secondary = gpd.read_file('data/catchments/catchments_secondary.shp')

Load data

In [3]:
businesses = pd.read_csv('data/Businesses.csv')
stops = pd.read_csv('data/Stops.txt')
polls = pd.read_csv('data/PollingPlaces2019.csv')
pops = pd.read_csv('data/Population.csv')
income = pd.read_csv('data/Income.csv')

# json file - sydney enrolment + school locations
enrol = pd.read_json('data/enrolment.json')


SQL Queries

In [4]:
from sqlalchemy import create_engine
from sqlalchemy import text
import psycopg2
import psycopg2.extras
import json
import pandas as pd

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [5]:
db, conn = pgconnect(credentials)

Connected successfully.


In [6]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [7]:
srid = 4283

# drop row if geometry is None
# SA2
sa2 = sa2[sa2.geometry.notnull()]
# polls
polls = polls[polls.the_geom.notnull()]
# enrolment
enrol = enrol.dropna(subset=['Latitude'])
enrol = enrol.dropna(subset=['Longitude'])

# convert geometry to WKTElement
# polls
polls['geom'] = gpd.points_from_xy(polls.longitude, polls.latitude)
polls['geom'] = polls['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

# enrolment
enrol['geom'] = gpd.points_from_xy(enrol.Longitude, enrol.Latitude)
enrol['geom'] = enrol['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

# convert polygon to multipolygon
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid=srid)

# convert geometry to WKTElement
sa2['geom'] = sa2['geometry'].apply(lambda x: create_wkt_element(x, srid))

# drop old geometry column
polls = polls.drop('the_geom', axis=1)
sa2 = sa2.drop('geometry', axis=1)

In [8]:
# subset sa2
sa2['sa2_code21'] = sa2['SA2_CODE21']
sa2['sa2_name21'] = sa2['SA2_NAME21']
sa2 = sa2[['sa2_code21', 
          'sa2_name21',
          'geom']]

In [9]:
# subset polling
polls = polls[['polling_place_name',
               'premises_name',
               'geom']].reset_index(drop=True)

In [10]:
# subset enrolment
enrol = enrol[['School_code',
               'School_name',
               'latest_year_enrolment_FTE',
               'geom']]

# change all variables to lowercase
enrol = enrol.rename(columns={'School_code': 'school_code',
                              'School_name': 'school_name',
                              'latest_year_enrolment_FTE': 'latest_year_enrolment_fte'})

# change variable types
enrol['school_code'] = enrol['school_code'].astype(int)
enrol['school_name'] = enrol['school_name'].astype(str)

# drop any value that is not a number from latest_year_enrolment_fte
enrol = enrol[enrol['latest_year_enrolment_fte'].str.strip().replace('', pd.NA).notna()]
enrol['latest_year_enrolment_fte'] = enrol['latest_year_enrolment_fte'].astype(float)

# drop missing values
enrol = enrol.dropna(subset=['school_code'])
enrol = enrol.dropna(subset=['school_name'])
enrol = enrol.dropna(subset=['latest_year_enrolment_fte'])
enrol = enrol.reset_index(drop=True)


In [11]:
enrol

,school_code,school_name,latest_year_enrolment_fte,geom
0,8473,Chifley College Senior Campus,471.0,POINT (150.829855 -33.769305)
1,8325,Moree Secondary College,389.0,POINT (149.842344 -29.46645)
2,8870,St Marys Senior High School,874.4,POINT (150.770024 -33.763119)
3,8374,Brisbane Water Secondary College Woy Woy Campus,663.4,POINT (151.323582 -33.492687)
4,8539,Sydney Secondary College Blackwattle Bay Campus,777.2,POINT (151.188357 -33.875142)
...,...,...,...,...
2165,4678,Ngarala Public School,168.0,POINT (151.069192 -33.78106)
2166,4682,Gregory Hills Public School,233.0,POINT (150.78 -34.025)
2167,4683,Murrumbateman Public School,107.0,POINT (149.030288 -34.9664751)
2168,4690,Jindabyne Public School,285.0,POINT (148.618556 -36.416783)


In [12]:
# create schema SA2
sql = """
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2 (
    sa2_code21 VARCHAR(20),
    sa2_name21 VARCHAR(150),
    geom GEOMETRY(MULTIPOLYGON, 4283)
);
"""

query(conn, sql)

Error encountered: 
This result object does not return rows. It has been closed automatically.


""


In [13]:
# create schema polling
sql = """
DROP TABLE IF EXISTS polls;
CREATE TABLE polls (
    polling_place_name VARCHAR(50),
    premises_name VARCHAR,
    latitude FLOAT,
    longitude FLOAT,
    geom GEOMETRY(POINT, 4283)
);
"""

query(conn, sql)

Error encountered: 
This result object does not return rows. It has been closed automatically.


""


In [14]:
# create schema enrolment
# create schema enrol
sql = """
DROP TABLE IF EXISTS enrol;
CREATE TABLE enrol (
    school_code INTEGER,
    school_name VARCHAR,
    latest_year_enrolment_fte FLOAT,
    geom GEOMETRY(Point, 4283)
);
"""

query(conn, sql)


Error encountered: 
This result object does not return rows. It has been closed automatically.


""


In [15]:
# SA2 input
sa2.to_sql('sa2', conn, if_exists='append', index=False, 
           dtype={'geom': Geometry('MULTIPOLYGON', srid)})

# polls nput 
polls.to_sql('polls', conn, if_exists='append', index=False, 
             dtype={'geom': Geometry('POINT', srid)})

# enrolment input
enrol.to_sql('enrol', conn, if_exists='append', index=False, 
             dtype={'geom': Geometry('POINT', srid)})

170

In [16]:
sql = """
select * from enrol
"""

query(conn, sql)

,school_code,school_name,latest_year_enrolment_fte,geom
0,8473,Chifley College Senior Campus,471.0,0101000020BB10000082AD122C8EDA62401CD3139678E2...
1,8325,Moree Secondary College,389.0,0101000020BB1000006B7F677BF4BA62401D3867446977...
2,8870,St Marys Senior High School,874.4,0101000020BB10000053245F09A4D8624066FA25E2ADE1...
3,8374,Brisbane Water Secondary College Woy Woy Campus,663.4,0101000020BB1000006072A3C85AEA624009151C5E10BF...
4,8539,Sydney Secondary College Blackwattle Bay Campus,777.2,0101000020BB100000205F420507E6624092AD2EA704F0...
...,...,...,...,...
2165,4678,Ngarala Public School,168.0,0101000020BB100000A22424D236E262405C1B2AC6F9E3...
2166,4682,Gregory Hills Public School,233.0,0101000020BB100000295C8FC2F5D86240333333333303...
2167,4683,Murrumbateman Public School,107.0,0101000020BB100000C32E8A1EF8A06240FC72C174B57B...
2168,4690,Jindabyne Public School,285.0,0101000020BB100000D4D7F335CB936240AF4335255935...


In [17]:
sql = """
select sa2.sa2_code21, 
sa2.sa2_name21, 
count(enrol.school_name) as school_count, 
sum(enrol.latest_year_enrolment_fte) as total_enrolment
from sa2 
left join enrol on ST_Intersects(enrol.geom, sa2.geom)
group by sa2.sa2_code21, sa2.sa2_name21
order by school_count desc;
"""

query(conn, sql)

,sa2_code21,sa2_name21,school_count,total_enrolment
0,112021249,Lismore Surrounds,17,999.2
1,113031271,Wagga Wagga Surrounds,13,967.2
2,110041205,Tamworth Surrounds,13,1043.2
3,103041079,Orange Surrounds,12,1123.0
4,113011257,Griffith Surrounds,12,1096.0
...,...,...,...,...
2449,302011026,McDowall,0,NaN
2450,302021027,Aspley,0,NaN
2451,302021028,Chermside,0,NaN
2452,302021029,Chermside West,0,NaN


In [19]:
# calculate z scores 
sql = """
WITH counts AS (
    SELECT 
        sa2.sa2_code21, 
        sa2.sa2_name21, 
        COUNT(enrol.school_name) AS school_count, 
        SUM(enrol.latest_year_enrolment_fte) AS total_enrolment
    FROM 
        sa2 
    LEFT JOIN 
        enrol ON ST_Intersects(enrol.geom, sa2.geom)
    GROUP BY 
        sa2.sa2_code21, sa2.sa2_name21
),
non_zero_counts AS (
    SELECT 
        sa2_code21, 
        sa2_name21, 
        school_count,
        total_enrolment
    FROM 
        counts
    WHERE 
        school_count > 0 AND total_enrolment > 0
),
z_scores AS (
    SELECT 
        sa2_code21, 
        sa2_name21, 
        school_count,
        total_enrolment,
        (school_count - AVG(school_count) OVER ()) / NULLIF(STDDEV(school_count) OVER (), 0) AS school_count_zscore,
        (total_enrolment - AVG(total_enrolment) OVER ()) / NULLIF(STDDEV(total_enrolment) OVER (), 0) AS total_enrolment_zscore
    FROM 
        non_zero_counts
)
SELECT 
    sa2_code21, 
    sa2_name21, 
    school_count_zscore,
    total_enrolment_zscore
FROM 
    z_scores
ORDER BY 
    school_count_zscore DESC;
"""

query(conn, sql)

,sa2_code21,sa2_name21,school_count_zscore,total_enrolment_zscore
0,112021249,Lismore Surrounds,5.840034,-0.344630
1,113031271,Wagga Wagga Surrounds,4.092990,-0.379844
2,110041205,Tamworth Surrounds,4.092990,-0.296212
3,101051540,Goulburn Surrounds,3.656229,-0.679820
4,113011257,Griffith Surrounds,3.656229,-0.238109
...,...,...,...,...
593,119021660,Campsie - South,-1.148141,-0.786121
594,124041468,Yarramundi - Londonderry,-1.148141,-1.215287
595,102011038,Saratoga - Davistown,-1.148141,-1.019411
596,119011655,Greenacre - North,-1.148141,-1.010607
